In [ ]:
from boto3.session import Session
import boto3
import scipy.io.wavfile as sciwav
from pydub import AudioSegment
import math
import os
import shutil
from io import BytesIO
from unsilence import Unsilence
import librosa
import os
import numpy as np
import sklearn
from sklearn import metrics
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

In [ ]:
ACCESS_KEY = ""
SECRET_KEY = ""
events_bucket_name = 'audiopipelineeventstest'
session = Session(aws_access_key_id=ACCESS_KEY,
              aws_secret_access_key=SECRET_KEY)


s3 = session.resource('s3')
events_bucket = s3.Bucket(events_bucket_name)

In [ ]:
S3_info = []
for s3_file in events_bucket.objects.all():
    #print(s3_file.key)
    object = s3.Object(events_bucket_name, s3_file.key)
    result = object.get()['Body'].read()
    wrapper = BytesIO(result)
    S3_info.append(wrapper)

In [ ]:
print(len(S3_info))

In [ ]:
original= []
for sample in S3_info:
    y, sr = librosa.load(sample)
    original.append((y,sr))

In [ ]:
mfcc = []
for elem in original:
    mfcc.append(librosa.feature.mfcc(elem[0],elem[1],n_mfcc=12))

In [ ]:
# mfcc

In [ ]:
mfcc_avg = []
for elem in mfcc:   
    avgFeatures = []
    for i in range(len(elem)):
        avgFeatures.append(numpy.mean(elem[i]))
    mfcc_avg.append(avgFeatures)

In [ ]:
#mfcc_avg

In [ ]:
X = StandardScaler().fit_transform(mfcc_avg)

In [ ]:
len(X)

In [ ]:
db = DBSCAN(eps=1.7).fit(X)

In [ ]:
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

In [ ]:
# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

In [ ]:
print("Silhouette Coefficient: %0.10f" % metrics.silhouette_score(X, labels))

In [ ]:
print(db.labels_)